In [ ]:
%matplotlib inline
import os
import torch
import random
import pickle
import functools
import numpy as np 
import pandas as pd 
from PIL import Image
import torch.nn as nn 
from pathlib import Path
import torch.nn.functional as F
from matplotlib import pyplot as plt 
from tqdm import tqdm, tqdm_notebook
from torchvision import transforms as T
import torch.utils.model_zoo as model_zoo
from torchvision.models.resnet import ResNet
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset, DataLoader
from torchvision.models.resnet import BasicBlock
from torchvision.models.resnet import Bottleneck
from pretrainedmodels.models.torchvision_models import pretrained_settings
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('../input/severstal-steel-defect-detection/train.csv')
d = {'ImageId': df['ImageId'].unique()}
train_df = pd.DataFrame(d)
train_df['ClassesList'] = ['' for i in range(len(train_df))]
train_df['EncodedPixels'] = ['' for i in range(len(train_df))]
for i in range(len(train_df)):
    train_df['ClassesList'][i] = df.loc[df['ImageId']==train_df['ImageId'][i]]['ClassId'].values.tolist()
    train_df['EncodedPixels'][i] = df.loc[df['ImageId']==train_df['ImageId'][i]]['EncodedPixels'].values.tolist()

In [ ]:
def make_mask( encoded, shape=(1600,256)):
    # Делим на два списка в соответствии с кодировкой
    if isinstance(encoded, str):
        encoded = list(map(int, encoded.split(' ')))
#     print(encoded)
    full,pixel,number = [],[],[]
    [pixel.append(encoded[i]) if i%2==0 else number.append(encoded[i]) for i in range(0, len(encoded))]
    # "Раскрываем" кодировку, получаем индексы закрашенных пикселей
    k=0
    for i in range(len(number)):
        for j in range(number[i]):
            ind = pixel[i]+j
            full.append(ind-1)
        k +=number[i]
    # Создаем массив под готовое изображение    
    mask = np.zeros((1600*256,1), dtype=int)
    # Закрашиваем соответствующие пиксели
    mask[full] = 255
    #преобразем к размерам фотографий металла
    res = np.reshape(mask,(1600, 256)).T
    res = Image.fromarray(res.astype(np.uint8))
    return res

In [ ]:
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

In [ ]:
# разные режимы датасета 
DATA_MODES = ['train', 'val', 'test']
# все изображения будут масштабированы к размеру 224x224 px
RESCALE_SIZE_1 = 800
RESCALE_SIZE_2 = 128
# работаем на видеокарте
DEVICE = torch.device("cuda")

In [ ]:
class SteelDataset(Dataset):
    def __init__(self, names, df, mode):
        super().__init__()
        self.names = names
        self.mode = mode
        if self.mode != 'test':
            self.df = df

        if self.mode not in DATA_MODES:
            print(f"{self.mode} is not correct; correct modes: {DATA_MODES}")
            raise NameError

        self.len_ = len(self.names)
        
    def __len__(self):
        return self.len_
      
    def load_sample(self, file, mode):
        if mode == 'test':
            image = Image.open('../input/severstal-steel-defect-detection/test_images/'+ file).convert("RGB")
        else:
            image = Image.open('../input/severstal-steel-defect-detection/train_images/'+ file).convert("RGB")
        image.load()
        return image
    
    def __getitem__(self, index):
            transforms_tens = T.Compose([
            T.ToTensor()])

            # загружаем и меняем размер изображения
            img = self._prepare_sample(self.load_sample(self.names[index], self.mode))

            if self.mode == 'test':
                img = np.array(img)
                max_value = 256 ** ((img.dtype == np.uint16) + 1) - 1
                img = (img / max_value).astype(np.float32)
                img = transforms_tens(img)
                return img
            
            # загружаем классы и маски
            labels = list(self.df['ClassesList'].loc[self.df['ImageId'] == self.names[index]])[0]
            mask = list(self.df['EncodedPixels'].loc[self.df['ImageId'] == self.names[index]])[0]
            num_obj = len(labels)
            masks = np.zeros((RESCALE_SIZE_2, RESCALE_SIZE_1, 4), dtype=np.float32)
            masks = np.transpose(masks,(2, 0, 1))
            for i in range(4):
                for j in range(num_obj):
                    if i == (labels[j]-1):
                # раскодирование масок
                        masks[i] = np.array(self._prepare_sample(make_mask(mask[j])))
                        masks[i] = masks[i] / 255

            # преобазование в тензоры
            masks = torch.as_tensor(masks, dtype=torch.float32)
            label = torch.as_tensor(labels)
            
            # преобразуем изображения и маски
            if self.mode == "train":
                img, masks= transforms_all(img,masks)
                
            img = np.array(img)
            max_value = 256 ** ((img.dtype == np.uint16) + 1) - 1
            img = (img / max_value).astype(np.float32)
            img = transforms_tens(img)
            
            return img, masks
        
    def _prepare_sample(self, image):
        image = image.resize((RESCALE_SIZE_1, RESCALE_SIZE_2))
        return image

In [ ]:
def transforms_all(image, segmentation):
    if random.random() > 0.5:
        image = TF.autocontrast(image)
    if random.random() > 0.5:
        image = TF.hflip(image)
        segmentation = TF.hflip(segmentation)
    if random.random() > 0.5:
        image = TF.vflip(image)
        segmentation = TF.vflip(segmentation)
    return image, segmentation

In [ ]:
from sklearn.model_selection import train_test_split
train_val_names = train_df['ImageId']
train_files,val_files = train_test_split(train_val_names, train_size=0.75)
val_dataset = SteelDataset(list(val_files), train_df,  mode='val')
train_dataset = SteelDataset(list(train_files),train_df, mode='train')

In [ ]:
def fit_epoch(model, train_loader, criterion, optimizer, batch_size):
    model.train()
    running_loss = 0.0
    dice = 0.0
    iou = 0.0
    for X_batch, Y_batch in train_loader:
        inputs = X_batch.to(DEVICE)
        labels = Y_batch.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs,labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.detach().cpu().numpy() 
        dice += dice_metric(outputs, labels)
    train_loss = running_loss / len(train_loader)
    train_dice = dice / len(train_loader)
    return train_loss, train_dice

In [ ]:
def eval_epoch(model, val_loader, criterion, batch_size):
    model.eval()
    dice = 0.0
    iou = 0.0
    running_loss = 0.0
    for X_batch, Y_batch in val_loader:
        inputs = X_batch.to(DEVICE)
        labels = Y_batch.to(DEVICE)

        with torch.no_grad():
            outputs = model(inputs)
            loss = criterion(outputs,labels.long())
            running_loss += loss.item()
            dice += dice_metric(outputs, labels)
    val_loss = running_loss / len(val_loader)
    val_dice = dice / len(val_loader)

    return val_loss, val_dice

In [ ]:
def train(train_files, val_files, model, epochs, batch_size):
    train_loader = DataLoader(train_dataset, batch_size=batch_size,num_workers = 2, shuffle=True)#,collate_fn = collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=batch_size,num_workers = 2, shuffle=False)#,collate_fn = collate_fn)
    history = []
    maxIoU = 0
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} train_dice: {t_dice:0.4f} \
    \nValidation  val_loss: {v_loss:0.4f} val_dice: {v_dice:0.4f} "
    criterion = nn.BCEWithLogitsLoss()
    
    with tqdm(desc="epoch", total=epochs) as pbar_outer:
        params = model.parameters()
        opt = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
        lr_scheduler = torch.optim.lr_scheduler.StepLR(opt,
                                                   step_size=20,
                                                   gamma=0.1)
        for epoch in range(epochs):
            train_loss, train_dice = fit_epoch(model, train_loader, criterion, opt, batch_size)
            lr_scheduler.step()    
            val_loss, val_dice = eval_epoch(model, val_loader, criterion, batch_size)
            history.append((train_loss, train_dice, val_loss, val_dice))
            if val_IoU > maxIoU:
                maxIoU = val_IoU
                torch.save(model, './weight_unet.dat')
            pbar_outer.update(1)
            tqdm.write(log_template.format(ep=epoch+1, t_loss=train_loss, t_dice = train_dice, \
                                           v_loss=val_loss, v_dice = val_acc ))   
    return history

In [ ]:
def dice_metric(probability, truth, threshold=0.5, reduction='none'):
    batch_size = len(truth)
    with torch.no_grad():
        probability = probability.view(batch_size, -1)
        truth = truth.view(batch_size, -1)
        assert(probability.shape == truth.shape)

        p = (probability > threshold).float()
        t = truth.float()

        t_sum = t.sum(-1)
        p_sum = p.sum(-1)
        neg_index = torch.nonzero(t_sum == 0)
        pos_index = torch.nonzero(t_sum >= 1)

        dice_neg = (p_sum == 0).float()
        dice_pos = 2 * (p*t).sum(-1)/((p+t).sum(-1))

        dice_neg = dice_neg[neg_index]
        dice_pos = dice_pos[pos_index]
        dice = dice_pos
    return dice

In [ ]:
def preprocess_input(x, mean=None, std=None, input_space='RGB', input_range=None, **kwargs):

    if input_space == 'BGR':
        x = x[..., ::-1].copy()

    if input_range is not None:
        if x.max() > 1 and input_range[1] == 1:
            x = x / 255.

    if mean is not None:
        mean = np.array(mean)
        x = x - mean

    if std is not None:
        std = np.array(std)
        x = x / std

    return x


class Model(nn.Module):

    def __init__(self):
        super().__init__()

    def initialize(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)


class Conv2dReLU(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding=0,
                 stride=1, use_batchnorm=True, **batchnorm_params):

        super().__init__()

        layers = [
            nn.Conv2d(in_channels, out_channels, kernel_size,
                              stride=stride, padding=padding, bias=not (use_batchnorm)),
            nn.ReLU(inplace=True),
        ]

        if use_batchnorm:
            layers.insert(1, nn.BatchNorm2d(out_channels, **batchnorm_params))

        self.block = nn.Sequential(*layers)

    def forward(self, x):
        return self.block(x)


class EncoderDecoder(Model):

    def __init__(self, encoder, decoder, activation):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

        if callable(activation) or activation is None:
            self.activation = activation
        elif activation == 'softmax':
            self.activation = nn.Softmax(dim=1)
        elif activation == 'sigmoid':
            self.activation = nn.Sigmoid()
        else:
            raise ValueError('Activation should be "sigmoid"/"softmax"/callable/None')

    def forward(self, x):
        """Sequentially pass `x` trough model`s `encoder` and `decoder` (return logits!)"""
        x = self.encoder(x)
        x = self.decoder(x)
        return x

    def predict(self, x):
        """Inference method. Switch model to `eval` mode, call `.forward(x)`
        and apply activation function (if activation is not `None`) with `torch.no_grad()`

        Args:
            x: 4D torch tensor with shape (batch_size, channels, height, width)

        Return:
            prediction: 4D torch tensor with shape (batch_size, classes, height, width)

        """
        if self.training:
            self.eval()

        with torch.no_grad():
            x = self.forward(x)
            if self.activation:
                x = self.activation(x)

        return x


class DecoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels, use_batchnorm=True):
        super().__init__()
        self.block = nn.Sequential(
            Conv2dReLU(in_channels, out_channels, kernel_size=3, padding=1, use_batchnorm=use_batchnorm),
            Conv2dReLU(out_channels, out_channels, kernel_size=3, padding=1, use_batchnorm=use_batchnorm),
        )

    def forward(self, x):
        x, skip = x
        x = F.interpolate(x, scale_factor=2, mode='nearest')
        if skip is not None:
            x = torch.cat([x, skip], dim=1)
        x = self.block(x)
        return x


class CenterBlock(DecoderBlock):

    def forward(self, x):
        return self.block(x)


class UnetDecoder(Model):

    def __init__(
            self,
            encoder_channels,
            decoder_channels=(256, 128, 64, 32, 16),
            final_channels=1,
            use_batchnorm=True,
            center=False,
    ):
        super().__init__()

        if center:
            channels = encoder_channels[0]
            self.center = CenterBlock(channels, channels, use_batchnorm=use_batchnorm)
        else:
            self.center = None

        in_channels = self.compute_channels(encoder_channels, decoder_channels)
        out_channels = decoder_channels

        self.layer1 = DecoderBlock(in_channels[0], out_channels[0], use_batchnorm=use_batchnorm)
        self.layer2 = DecoderBlock(in_channels[1], out_channels[1], use_batchnorm=use_batchnorm)
        self.layer3 = DecoderBlock(in_channels[2], out_channels[2], use_batchnorm=use_batchnorm)
        self.layer4 = DecoderBlock(in_channels[3], out_channels[3], use_batchnorm=use_batchnorm)
        self.layer5 = DecoderBlock(in_channels[4], out_channels[4], use_batchnorm=use_batchnorm)
        self.final_conv = nn.Conv2d(out_channels[4], final_channels, kernel_size=(1, 1))

        self.initialize()

    def compute_channels(self, encoder_channels, decoder_channels):
        channels = [
            encoder_channels[0] + encoder_channels[1],
            encoder_channels[2] + decoder_channels[0],
            encoder_channels[3] + decoder_channels[1],
            encoder_channels[4] + decoder_channels[2],
            0 + decoder_channels[3],
        ]
        return channels

    def forward(self, x):
        encoder_head = x[0]
        skips = x[1:]

        if self.center:
            encoder_head = self.center(encoder_head)

        x = self.layer1([encoder_head, skips[0]])
        x = self.layer2([x, skips[1]])
        x = self.layer3([x, skips[2]])
        x = self.layer4([x, skips[3]])
        x = self.layer5([x, None])
        x = self.final_conv(x)

        return x


class ResNetEncoder(ResNet):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.pretrained = False
        del self.fc

    def forward(self, x):
        x0 = self.conv1(x)
        x0 = self.bn1(x0)
        x0 = self.relu(x0)

        x1 = self.maxpool(x0)
        x1 = self.layer1(x1)

        x2 = self.layer2(x1)
        x3 = self.layer3(x2)
        x4 = self.layer4(x3)

        return [x4, x3, x2, x1, x0]

    def load_state_dict(self, state_dict, **kwargs):
        state_dict.pop('fc.bias')
        state_dict.pop('fc.weight')
        super().load_state_dict(state_dict, **kwargs)


resnet_encoders = {
    'resnet18': {
        'encoder': ResNetEncoder,
        'pretrained_settings': pretrained_settings['resnet18'],
        'out_shapes': (512, 256, 128, 64, 64),
        'params': {
            'block': BasicBlock,
            'layers': [2, 2, 2, 2],
        },
    },
    
    'resnet50': {
        'encoder': ResNetEncoder,
        'pretrained_settings': pretrained_settings['resnet50'],
        'out_shapes': (2048, 1024, 512, 256, 64),
        'params': {
            'block': Bottleneck,
            'layers': [3, 4, 6, 3],
        },
    },
}

encoders = {}
encoders.update(resnet_encoders)

def get_encoder(name, encoder_weights=None):
    Encoder = encoders[name]['encoder']
    encoder = Encoder(**encoders[name]['params'])
    encoder.out_shapes = encoders[name]['out_shapes']

    if encoder_weights is not None:
        settings = encoders[name]['pretrained_settings'][encoder_weights]
        encoder.load_state_dict(model_zoo.load_url(settings['url']))

    return encoder


def get_encoder_names():
    return list(encoders.keys())


def get_preprocessing_fn(encoder_name, pretrained='imagenet'):
    settings = encoders[encoder_name]['pretrained_settings']

    if pretrained not in settings.keys():
        raise ValueError('Avaliable pretrained options {}'.format(settings.keys()))

    input_space = settings[pretrained].get('input_space')
    input_range = settings[pretrained].get('input_range')
    mean = settings[pretrained].get('mean')
    std = settings[pretrained].get('std')
    
    return functools.partial(preprocess_input, mean=mean, std=std, input_space=input_space, input_range=input_range)


class Unet(EncoderDecoder):

    def __init__(
            self,
            encoder_name='resnet34',
            encoder_weights='imagenet',
            decoder_use_batchnorm=True,
            decoder_channels=(256, 128, 64, 32, 16),
            classes=1,
            activation='sigmoid',
            center=False,  # usefull for VGG models
    ):
        encoder = get_encoder(
            encoder_name,
            encoder_weights=encoder_weights
        )

        decoder = UnetDecoder(
            encoder_channels=encoder.out_shapes,
            decoder_channels=decoder_channels,
            final_channels=classes,
            use_batchnorm=decoder_use_batchnorm,
            center=center,
        )

        super().__init__(encoder, decoder, activation)

In [ ]:
model = Unet("resnet18", encoder_weights="imagenet", classes=4, activation=None)
# model = Unet("resnet50", encoder_weights="imagenet", classes=4, activation=None)

In [ ]:
history = train(train_dataset, val_dataset, model, epochs = 40, batch_size = 5)

In [ ]:
train_loss, train_acc, val_loss, val_acc = zip(*history)

In [ ]:
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20,13))
    ax1.plot(train_loss, label='train', marker='o')
    ax1.plot(val_loss, label='val', marker='o')
    ax1.set_title('Loss per epoch')
    ax1.set_ylabel('loss');
    ax1.set_xlabel('epoch')
    ax1.legend(), ax1.grid()
    
    ax2.plot(train_dice, label='train_dice', marker='*')
    ax2.plot(val_dice, label='val_dice',  marker='*')
    ax2.set_title('Score per epoch')
    ax2.set_ylabel('mean dice')
    ax2.set_xlabel('epoch')
    ax2.legend(), ax2.grid()
    
#     plt.savefig('0.710615.png')
    plt.show()

In [ ]:
def predict_image_mask_miou(model, image, mask):
    model.eval()
    
    image = image.to(DEVICE)
    mask = mask.to(DEVICE)
    with torch.no_grad():
        
        image = image.unsqueeze(0)
        mask = mask.unsqueeze(0)
        
        output = model(image)
        print(output)
        score = dice_metric(output, mask)
        masked = torch.argmax(output, dim=1)
        masked = masked.cpu().squeeze(0)
    return masked, score

In [ ]:
def miou_score(model, val_dataset):
    score_iou = []
    for i in tqdm(range(len(val_dataset))):
        img, mask = val_dataset[i]
        pred_mask, score = predict_image_mask_miou(model, img, mask)
        score_iou.append(score)
    return score_iou

In [ ]:
# model = torch.load('../input/unet-weight/weight_unet_wew.dat')

In [ ]:
print(np.mean(miou_score(model, val_dataset)))